# Scrape

In [1]:
from selenium import webdriver
from selenium.webdriver.support.ui import WebDriverWait
from selenium.webdriver.support import expected_conditions as EC
from selenium.webdriver.common.by import By
from selenium.common.exceptions import TimeoutException
from selenium.common.exceptions import NoSuchElementException

from bs4 import BeautifulSoup
import lxml
import pandas as pd
import time
import urllib.request
import re

In [2]:
zipdf = pd.read_csv("ZIpcodes.csv")
zipdf["Zipcode"] = zipdf["Zipcode"].astype(str)
zipdf["Zipcode"] = zipdf["Zipcode"].str[:-2]
zipdf = zipdf["Zipcode"].values.tolist()

# Rent Cost Crawler

In [5]:
prices_list = []
titles_list = []
dates_list = []
address_list = []
bedrooms_list = []
sqfts_list = []
url_list = []
zipcode_list = []

class CraigslistRENTScraper(object):
    def __init__(self, location, miles, zipcode, bedrooms, sqft):
# What information I'm scrapping
        self.location = location
        self.miles = miles
        self.bedrooms = bedrooms
        self.sqft = sqft
        self.zipcode = zipcode
        
        self.driver = webdriver.Firefox()
        self.delay = 3


# Launch URL and confirm it fully loaded            
    def grab_data(self):
        all_posts = self.driver.find_elements_by_class_name("result-row")
        for post in all_posts:
            title = post.text.split("$")

            if title[0] == '':
                title = title[1]
            else:
                title = title[0]

            info = post.text.split(" - ")
            address = info[-1]
            address = address.strip("()")

            bedroom = post.text
            pattern = r"\s\d*[b][r]\s"
            bedroom = (re.findall(pattern,bedroom)[0])

            sqft = post.text
            sqftpat = r"\s\d*[f][t][2]"
            sqft = (re.findall(sqftpat,sqft)[0])

            title = title.split("\n")
            price = title[0]
            title = title[-1]

            title = title.split(" ")

            month = title[0]
            day = title[1]
            title = ' '.join(title[2:])
            date = month + " " + day
            
            zipcode = self.url
            pattern = r"[t][a][l][=]\d*"
            zipcode = (re.findall(pattern,zipcode)[0])
# Add to Lists        
            prices_list.append(price)
            titles_list.append(title)
            dates_list.append(date)
            address_list.append(address)
            bedrooms_list.append(bedroom)
            sqfts_list.append(sqft)
            zipcode_list.append(zipcode)


        return prices_list, titles_list, dates_list, address_list, bedrooms_list, sqfts_list
            
    
    def load_craigslist_url(self):      
        for zipcode in zipdf:
            for i in range(0,24):
                    if i != 0:
                        self.url = f"https://{location}.craigslist.org/search/apa?search_distance={miles}&postal={zipcode}&availabilityMode=0&minSqft={sqft}&min_bedrooms={bedrooms}&s=" + str(i * 120)
                        print(zipcode)
                        self.driver.get(self.url)
                        wait = WebDriverWait(self.driver, self.delay)
                        wait.until(EC.presence_of_element_located((By.ID, "searchform")))
                        print("Page is ready")
                        print("s=" + str(i * 120))
                        scraper.extract_post_urls()
                        scraper.grab_data()                
                        try:
                            self.driver.find_element_by_link_text('next >')
                        except:
                            break
 
                    else:
                        self.url = f"https://{location}.craigslist.org/search/apa?search_distance={miles}&postal={zipcode}&availabilityMode=0&minSqft={sqft}&min_bedrooms={bedrooms}"
                        self.driver.get(self.url)
                        print("First iteration")
                        wait = WebDriverWait(self.driver, self.delay)
                        wait.until(EC.presence_of_element_located((By.ID, "searchform")))
                        print("Page is ready")
                        print(zipcode)
                        scraper.extract_post_urls()
                        scraper.grab_data()
                        try:
                            self.driver.find_element_by_link_text('next >')
                        except:
                            break
    
                            
# Collect post urls and build list    
    def extract_post_urls(self):
        html_page = urllib.request.urlopen(self.url)
        soup = BeautifulSoup(html_page, "lxml")
        for link in soup.findAll("a", {"class": "result-title hdrlnk"}):
            url_list.append(link["href"])
        return url_list
        
    def quit(self):
        self.driver.close()
    
# Specifications for scrape  
location = "Portland"
miles = "6"
bedrooms = "1"
sqft = "1"
zipcode = ""
# add price

scraper = CraigslistRENTScraper(location, miles, zipcode, bedrooms, sqft)
scraper.load_craigslist_url()
prices, titles, dates, address, bedrooms, sqfts = scraper.grab_data()
url_list = scraper.extract_post_urls()
scraper.quit()


First iteration
Page is ready
97201


ConnectionAbortedError: [WinError 10053] An established connection was aborted by the software in your host machine

In [31]:
# print(len(prices_list))
# print(len(dates_list))
# print(len(bedrooms_list))
# print(len(sqfts_list))
# print(len(address_list))
print(len(url_list))
print(len(zipcode_list))

16992
16953


In [12]:
rentdf = pd.DataFrame(data = {
    "Prices" : prices_list,
    "Title" : titles_list,
    "Date" : dates_list,
    "Address" : address_list,
    "Bedrooms" : bedrooms_list,
    "Sqft" : sqfts_list,
#     "URL" : url_list,
    "Zipcode" : zipcode_list
})

In [15]:
rentdf["rooms"] = rentdf["Bedrooms"].str[:2]
rentdf["Square"] = rentdf["Sqft"].str[:-3]
rentdf["Zipcode"] = rentdf["Zipcode"].str[4:]
# rentdf["Location"]

In [16]:
rentdf.head()

,Address,Bedrooms,Date,Prices,Sqft,Title,Zipcode,rooms,Square
0,SE 26th & Hawthorne,1br,Aug 10,1500,605ft2,Top Floor One Bedroom- Right in the center of ...,97201,1,605
1,"6306 SW 30th Ave Portland, OR",3br,Aug 10,3095,2316ft2,"granite counters, tons of storage, stainless s...",97201,3,2316
2,"SE portland, Sellwood, Brooklyn, Moreland",2br,Aug 10,1850,936ft2,Overlook the Willamette from a Two-Bedroom Apt...,97201,2,936
3,Portland,1br,Aug 10,1265,668ft2,THIS IS THE ONE (bedroom apartment) FOR YOU!!!,97201,1,668
4,Ladd's Addition,2br,Aug 10,1895,893ft2,Beautiful and NEW Town Home - Ladd's Addition ...,97201,2,893


In [18]:
rent_df = rentdf.drop_duplicates(["Title", "Prices", "Sqft"], keep='first', inplace=False)

In [19]:
rent_df.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 10237 entries, 0 to 37467
Data columns (total 9 columns):
Address     10237 non-null object
Bedrooms    10237 non-null object
Date        10237 non-null object
Prices      10237 non-null object
Sqft        10237 non-null object
Title       10237 non-null object
Zipcode     10237 non-null object
rooms       10237 non-null object
Square      10237 non-null object
dtypes: object(9)
memory usage: 799.8+ KB


In [20]:
rent_df.to_csv("Oregon Rental.csv")

In [28]:
zipcodes = rentdf["Zipcode"]

In [23]:
rent_df.head()

,Address,Bedrooms,Date,Prices,Sqft,Title,Zipcode,rooms,Square
0,SE 26th & Hawthorne,1br,Aug 10,1500,605ft2,Top Floor One Bedroom- Right in the center of ...,97201,1,605
1,"6306 SW 30th Ave Portland, OR",3br,Aug 10,3095,2316ft2,"granite counters, tons of storage, stainless s...",97201,3,2316
2,"SE portland, Sellwood, Brooklyn, Moreland",2br,Aug 10,1850,936ft2,Overlook the Willamette from a Two-Bedroom Apt...,97201,2,936
3,Portland,1br,Aug 10,1265,668ft2,THIS IS THE ONE (bedroom apartment) FOR YOU!!!,97201,1,668
4,Ladd's Addition,2br,Aug 10,1895,893ft2,Beautiful and NEW Town Home - Ladd's Addition ...,97201,2,893


In [24]:
zippy = zipcodes.unique()
zippy = pd.DataFrame(zippy)
zippy.to_csv("zippy.csv")


In [25]:
len(zippy)

149

In [27]:
rentdf.info()


<class 'pandas.core.frame.DataFrame'>
RangeIndex: 38548 entries, 0 to 38547
Data columns (total 9 columns):
Address     38548 non-null object
Bedrooms    38548 non-null object
Date        38548 non-null object
Prices      38548 non-null object
Sqft        38548 non-null object
Title       38548 non-null object
Zipcode     38548 non-null object
rooms       38548 non-null object
Square      38548 non-null object
dtypes: object(9)
memory usage: 2.6+ MB


# Craigslist Real Estate Crawler

In [48]:
Hprices_list = []
Htitles_list = []
Hdates_list = []
Haddress_list = []
Hbedrooms_list = []
Hsqfts_list = []
Hurl_list = []
Hzipcode_list = []

class CraigslistHOUSEScraper(object):
    def __init__(self, location, miles, zipcode, bedrooms, sqft):
# What information I'm scrapping
        self.location = location
        self.miles = miles
        self.bedrooms = bedrooms
        self.sqft = sqft
        self.zipcode = zipcode
        
        self.driver = webdriver.Firefox()
        self.delay = 3


# Launch URL and confirm it fully loaded            
    def grab_data(self):
        Hall_posts = self.driver.find_elements_by_class_name("result-row")
        for post in Hall_posts:
            title = post.text.split("$")

            if title[0] == '':
                title = title[1]
            else:
                title = title[0]

            info = post.text.split(" - ")
            address = info[-1]
            address = address.strip("()")

            bedroom = post.text
            pattern = r"\s\d*[b][r]\s"
            bedroom = (re.findall(pattern,bedroom)[0])

            sqft = post.text
            sqftpat = r"\s\d*[f][t][2]"
            sqft = (re.findall(sqftpat,sqft)[0])

            title = title.split("\n")
            price = title[0]
            title = title[-1]

            title = title.split(" ")

            month = title[0]
            day = title[1]
            title = ' '.join(title[2:])
            date = month + " " + day
            
            zipcode = self.url
            pattern = r"[t][a][l][=]\d*"
            zipcode = (re.findall(pattern,zipcode)[0])
# Add to Lists        
            Hprices_list.append(price)
            Htitles_list.append(title)
            Hdates_list.append(date)
            Haddress_list.append(address)
            Hbedrooms_list.append(bedroom)
            Hsqfts_list.append(sqft)
            Hzipcode_list.append(zipcode)


        return Hprices_list, Htitles_list, Hdates_list, Haddress_list, Hbedrooms_list, Hsqfts_list
            
    
    def load_craigslist_url(self):      
        for zipcode in zipdf:
            for i in range(0,24):
                    if i != 0:
                        self.url = f"https://{location}.craigslist.org/search/rea?search_distance={miles}&postal={zipcode}&availabilityMode=0&minSqft={sqft}&min_bedrooms={bedrooms}&s=" + str(i * 120)
                        print(zipcode)
                        self.driver.get(self.url)
                        wait = WebDriverWait(self.driver, self.delay)
                        wait.until(EC.presence_of_element_located((By.ID, "searchform")))
                        print("Page is ready")
                        print("s=" + str(i * 120))
                        scraper.extract_post_urls()
                        scraper.grab_data()                
                        try:
                            self.driver.find_element_by_link_text('next >')
                        except:
                            break
 
                    else:
                        self.url = f"https://{location}.craigslist.org/search/rea?search_distance={miles}&postal={zipcode}&availabilityMode=0&minSqft={sqft}&min_bedrooms={bedrooms}"
                        self.driver.get(self.url)
                        print("First iteration")
                        wait = WebDriverWait(self.driver, self.delay)
                        wait.until(EC.presence_of_element_located((By.ID, "searchform")))
                        print("Page is ready")
                        print(zipcode)
                        scraper.extract_post_urls()
                        scraper.grab_data()
                        try:
                            self.driver.find_element_by_link_text('next >')
                        except:
                            break
    
                            
# Collect post urls and build list    
    def extract_post_urls(self):
        html_pages = urllib.request.urlopen(self.url)
        soup = BeautifulSoup(html_pages, "lxml")
        for link in soup.findAll("a", {"class": "result-title hdrlnk"}):
            Hurl_list.append(link["href"])
        return Hurl_list
        
    def quit(self):
        self.driver.close()
    
# Specifications for scrape  
location = "Portland"
miles = "7"
bedrooms = "1"
sqft = "1"


scraper = CraigslistHOUSEScraper(location, miles, zipcode, bedrooms, sqft)
scraper.load_craigslist_url()
prices, titles, dates, address, bedrooms, sqfts = scraper.grab_data()
Hurl_list = scraper.extract_post_urls()
scraper.quit()

First iteration
Page is ready
97201
97201
Page is ready
s=120
97201
Page is ready
s=240
97201
Page is ready
s=360
97201
Page is ready
s=480
97201
Page is ready
s=600
97201
Page is ready
s=720
First iteration
Page is ready
97301
97301
Page is ready
s=120
First iteration
Page is ready
97401
97401
Page is ready
s=120
97401
Page is ready
s=240
First iteration
Page is ready
97030
97030
Page is ready
s=120
97030
Page is ready
s=240
First iteration
Page is ready
97123
97123
Page is ready
s=120
First iteration
Page is ready
97005
97005
Page is ready
s=120
97005
Page is ready
s=240
97005
Page is ready
s=360
97005
Page is ready
s=480
97005
Page is ready
s=600
First iteration
Page is ready
97701
97701
Page is ready
s=120
97701
Page is ready
s=240
First iteration
Page is ready
97501
First iteration
Page is ready
97477
97477
Page is ready
s=120
97477
Page is ready
s=240
First iteration
Page is ready
97333
First iteration
Page is ready
97322
First iteration
Page is ready
97281
First iteration
Page i

First iteration
Page is ready
97346
First iteration
Page is ready
97830
First iteration
Page is ready
97040
First iteration
Page is ready
97137
First iteration
Page is ready
97907
First iteration
Page is ready
97867
First iteration
Page is ready
97522
First iteration
Page is ready
97037
First iteration
Page is ready
97147
First iteration
Page is ready
97833
First iteration
Page is ready
97623
First iteration
Page is ready
97202
97202
Page is ready
s=120
97202
Page is ready
s=240
97202
Page is ready
s=360
97202
Page is ready
s=480
97202
Page is ready
s=600
97202
Page is ready
s=720
First iteration
Page is ready
97203
97203
Page is ready
s=120
97203
Page is ready
s=240
97203
Page is ready
s=360
First iteration
Page is ready
97204
97204
Page is ready
s=120
97204
Page is ready
s=240
97204
Page is ready
s=360
97204
Page is ready
s=480
97204
Page is ready
s=600
First iteration
Page is ready
97205
97205
Page is ready
s=120
97205
Page is ready
s=240
97205
Page is ready
s=360
97205
Page is read

97271
Page is ready
s=1080
97271
Page is ready
s=1200
97271
Page is ready
s=1320
97271
Page is ready
s=1440
97271
Page is ready
s=1560
97271
Page is ready
s=1680
97271
Page is ready
s=1800
97271
Page is ready
s=1920
97271
Page is ready
s=2040
97271
Page is ready
s=2160
97271
Page is ready
s=2280
97271
Page is ready
s=2400
97271
Page is ready
s=2520
97271
Page is ready
s=2640
First iteration
Page is ready
97272
97272
Page is ready
s=120
97272
Page is ready
s=240
97272
Page is ready
s=360
97272
Page is ready
s=480
97272
Page is ready
s=600
97272
Page is ready
s=720
97272
Page is ready
s=840
97272
Page is ready
s=960
97272
Page is ready
s=1080
97272
Page is ready
s=1200
97272
Page is ready
s=1320
97272
Page is ready
s=1440
97272
Page is ready
s=1560
97272
Page is ready
s=1680
97272
Page is ready
s=1800
97272
Page is ready
s=1920
97272
Page is ready
s=2040
97272
Page is ready
s=2160
97272
Page is ready
s=2280
97272
Page is ready
s=2400
97272
Page is ready
s=2520
97272
Page is ready
s=2640


In [59]:
Housedf = pd.DataFrame(data = {
    "Prices" : Hprices_list,
    "Title" : Htitles_list,
    "Date" : Hdates_list,
    "Address" : Haddress_list,
    "Bedrooms" : Hbedrooms_list,
    "Sqft" : Hsqfts_list,
#     "URL" : Hurl_list,
    "Zipcode" : Hzipcode_list
})

In [71]:
Housedf["Rooms"] = Housedf["Bedrooms"].str[:2]
Housedf["Square"] = Housedf["Sqft"].str[:-3]
Housedf["zipcode"] = Housedf["Zipcode"].str[4:]

In [29]:
Housedf.head()

NameError: name 'Housedf' is not defined

In [73]:
Housedf1.head()

,URL
0,https://portland.craigslist.org/mlt/reb/d/wow-...
1,https://portland.craigslist.org/mlt/reb/d/open...
2,https://portland.craigslist.org/mlt/reb/d/char...
3,https://portland.craigslist.org/mlt/reo/d/home...
4,https://portland.craigslist.org/mlt/reb/d/home...


In [74]:
Housedf1.to_csv("HouseURL.csv")
Housedf.to_csv("Housedata.csv")

In [66]:
Housedf1 = pd.DataFrame(data = {
    "URL" : Hurl_list
})

In [52]:
H = Htitles_list.unique()
H = pd.DataFrame(zippy)


AttributeError: 'list' object has no attribute 'unique'